**Data Visualization**

In [0]:
#Word Cloud
from wordcloud import WordCloud
train_q = pd.Series(df['question1'].tolist() + df['question2'].tolist()).astype(str)
cloud = WordCloud(width=1440, height=1080).generate(" ".join(train_q.astype(str)))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off')

In [0]:
#Loading the pickle file with the extracted features
df4= pd.read_pickle('/content/drive/My Drive/Final_DF_question_pair.pkl')

In [0]:
#Comparing common words between the question pairs and word share as features
plt.subplot(1,2,1)
sns.distplot(df4[df4['is_duplicate'] == 1.0]['word_Common'][0:] , label = "1", color = 'red')
sns.distplot(df4[df4['is_duplicate'] == 0.0]['word_Common'][0:] , label = "0" , color = 'blue' )


plt.subplot(1,2,2)
sns.distplot(df4[df4['is_duplicate'] == 1.0]['word_share'][0:] , label = "1", color = 'red')
sns.distplot(df4[df4['is_duplicate'] == 0.0]['word_share'][0:] , label = "0" , color = 'blue' )
y.label()
plt.show()

In [0]:
#PairPlot
import seaborn as sns
n = 10000
sns.pairplot(df4[['len_q1','len_q2', 'diff_len', 'len_char_q1', 'len_char_q2',
                  'len_word_q1','len_word_q2', 'common_words','is_duplicate']][0:n],hue= 'is_duplicate')

**Subset Model Implementation**

**Logistic Regression**

Basic Features

In [0]:
X1 = df4.iloc[:,15:22]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#logistic regression with hyperparameter tuning:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X1_train, y1_train)
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X1_train, y1_train)

# predict using test set
# X_test_features contains all features from feature sets 1,2 & 3 for the test set question-pairs
logr_pred = logr_model.predict(X1_test)
C = confusion_matrix(y1_test, logr_pred,labels = [1,0])

In [0]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
average_precision = average_precision_score(y1_test, logr_pred)
print('confusion_matrix',C)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
score =accuracy_score(y1_test,logr_pred)
print("Accuracy of Logistic Regression:",score)

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature

precision, recall, _ = precision_recall_curve(y1_test, logr_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

Word2Vec Embeddings

In [0]:
X1 = df4.iloc[:,1:14]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#logistic regression with hyperparameter tuning:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X1_train, y1_train)
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X1_train, y1_train)

# predict using test set
# X_test_features contains all features from feature sets 1,2 & 3 for the test set question-pairs
logr_pred = logr_model.predict(X1_test)
C = confusion_matrix(y1_test, logr_pred,labels = [1,0])

In [0]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
average_precision = average_precision_score(y1_test, logr_pred)
print('confusion_matrix',C)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
score =accuracy_score(y1_test,logr_pred)
print("Accuracy of Logistic Regression:",score)

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature

precision, recall, _ = precision_recall_curve(y1_test, logr_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

Basic+Word2Vec

In [0]:
X1 = df4.iloc[:,1:22]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#logistic regression with hyperparameter tuning:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X1_train, y1_train)
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X1_train, y1_train)

# predict using test set
# X_test_features contains all features from feature sets 1,2 & 3 for the test set question-pairs
logr_pred = logr_model.predict(X1_test)
C = confusion_matrix(y1_test, logr_pred,labels = [1,0])

In [0]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
average_precision = average_precision_score(y1_test, logr_pred)
print('confusion_matrix',C)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
score =accuracy_score(y1_test,logr_pred)
print("Accuracy of Logistic Regression:",score)

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature

precision, recall, _ = precision_recall_curve(y1_test, logr_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

Fuzzy Features

In [0]:
X1 = df4.iloc[:,23:31]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#logistic regression with hyperparameter tuning:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X1_train, y1_train)
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X1_train, y1_train)

# predict using test set
# X_test_features contains all features from feature sets 1,2 & 3 for the test set question-pairs
logr_pred = logr_model.predict(X1_test)
C = confusion_matrix(y1_test, logr_pred,labels = [1,0])

In [0]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
average_precision = average_precision_score(y1_test, logr_pred)
print('confusion_matrix',C)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
score =accuracy_score(y1_test,logr_pred)
print("Accuracy of Logistic Regression:",score)

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature

precision, recall, _ = precision_recall_curve(y1_test, logr_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

Fuzzy+Basic

In [0]:
X1 = df4.iloc[:,15:31]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#logistic regression with hyperparameter tuning:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X1_train, y1_train)
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X1_train, y1_train)

# predict using test set
# X_test_features contains all features from feature sets 1,2 & 3 for the test set question-pairs
logr_pred = logr_model.predict(X1_test)
C = confusion_matrix(y1_test, logr_pred,labels = [1,0])

In [0]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
average_precision = average_precision_score(y1_test, logr_pred)
print('confusion_matrix',C)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
score =accuracy_score(y1_test,logr_pred)
print("Accuracy of Logistic Regression:",score)

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature

precision, recall, _ = precision_recall_curve(y1_test, logr_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

Fuzzy + Word2Vev Embeddings

In [0]:
rng = list(range(1,15)) + list(range(23,30))
X1 = df4.iloc[:,rng]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#logistic regression with hyperparameter tuning:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X1_train, y1_train)
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X1_train, y1_train)

# predict using test set
# X_test_features contains all features from feature sets 1,2 & 3 for the test set question-pairs
logr_pred = logr_model.predict(X1_test)
C = confusion_matrix(y1_test, logr_pred,labels = [1,0])

In [0]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
average_precision = average_precision_score(y1_test, logr_pred)
print('confusion_matrix',C)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
score =accuracy_score(y1_test,logr_pred)
print("Accuracy of Logistic Regression:",score)

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature

precision, recall, _ = precision_recall_curve(y1_test, logr_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

**Random Forest**

In [0]:
X1 = df4.iloc[:,15:22]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier as RFC
estimators = [100,150,200,300,400,600,800]

test_scores = []
train_scores = []
for i in estimators:
    clf = RFC(n_estimators=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('estimators = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:

Depth = [5,10,12,15,20,25,50]
test_scores = []
train_scores = []
for i in Depth:
    clf = RFC(n_estimators=100,max_depth=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('Depth = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:
#optimum depth = 11, optimum components = 400:
clf = RFC(n_estimators=400,max_depth=11,n_jobs=-1)
clf.fit(X1_train,y1_train)
y_pred_rf = clf.predict(X1_test)

In [0]:
confusion_matrix_rf = confusion_matrix(y1_test, y_pred_rf,labels = [1,0])
print(confusion_matrix_rf)

accuracy_rf = accuracy_score(y1_test,y_pred_rf)
print(accuracy_rf)

In [0]:
# RANDOM FOREST METRICS:- plot the confusion matrix, Average precision Recall, F1 score-
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from inspect import signature


average_precision = average_precision_score(y1_test,y_pred_rf)    
f1_score_rf = f1_score(y1_test,y_pred_rf)
print("F1 Score of Random Forest classifier is:",f1_score_rf)
print("Confusion Matrix for Random Forest is:",confusion_matrix_rf)
print("precision score:",precision_score(y1_test, y_pred_rf))
print("Recall:",recall_score(y1_test,y_pred_rf))
print("Accuracy:",accuracy_rf)
print("Average Precision Recall Score:",average_precision)


Word2Vec Embeddings

In [0]:
X1 = df4.iloc[:,1:14]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier as RFC
estimators = [100,150,200,300,400,600,800]

test_scores = []
train_scores = []
for i in estimators:
    clf = RFC(n_estimators=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('estimators = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:

Depth = [5,10,12,15,20,25,50]
test_scores = []
train_scores = []
for i in Depth:
    clf = RFC(n_estimators=100,max_depth=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('Depth = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:
#optimum depth = 11, optimum components = 400:
clf = RFC(n_estimators=400,max_depth=11,n_jobs=-1)
clf.fit(X1_train,y1_train)
y_pred_rf = clf.predict(X1_test)

In [0]:
confusion_matrix_rf = confusion_matrix(y1_test, y_pred_rf,labels = [1,0])
print(confusion_matrix_rf)

accuracy_rf = accuracy_score(y1_test,y_pred_rf)
print(accuracy_rf)

In [0]:
# RANDOM FOREST METRICS:- plot the confusion matrix, Average precision Recall, F1 score-
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from inspect import signature


average_precision = average_precision_score(y1_test,y_pred_rf)    
f1_score_rf = f1_score(y1_test,y_pred_rf)
print("F1 Score of Random Forest classifier is:",f1_score_rf)
print("Confusion Matrix for Random Forest is:",confusion_matrix_rf)
print("precision score:",precision_score(y1_test, y_pred_rf))
print("Recall:",recall_score(y1_test,y_pred_rf))
print("Accuracy:",accuracy_rf)
print("Average Precision Recall Score:",average_precision)


Basic + Word2Vec Embeddings

In [0]:
X1 = df4.iloc[:,1:22]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier as RFC
estimators = [100,150,200,300,400,600,800]

test_scores = []
train_scores = []
for i in estimators:
    clf = RFC(n_estimators=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('estimators = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:

Depth = [5,10,12,15,20,25,50]
test_scores = []
train_scores = []
for i in Depth:
    clf = RFC(n_estimators=100,max_depth=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('Depth = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:
#optimum depth = 11, optimum components = 400:
clf = RFC(n_estimators=400,max_depth=11,n_jobs=-1)
clf.fit(X1_train,y1_train)
y_pred_rf = clf.predict(X1_test)

In [0]:
confusion_matrix_rf = confusion_matrix(y1_test, y_pred_rf,labels = [1,0])
print(confusion_matrix_rf)

accuracy_rf = accuracy_score(y1_test,y_pred_rf)
print(accuracy_rf)

In [0]:
# RANDOM FOREST METRICS:- plot the confusion matrix, Average precision Recall, F1 score-
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from inspect import signature


average_precision = average_precision_score(y1_test,y_pred_rf)    
f1_score_rf = f1_score(y1_test,y_pred_rf)
print("F1 Score of Random Forest classifier is:",f1_score_rf)
print("Confusion Matrix for Random Forest is:",confusion_matrix_rf)
print("precision score:",precision_score(y1_test, y_pred_rf))
print("Recall:",recall_score(y1_test,y_pred_rf))
print("Accuracy:",accuracy_rf)
print("Average Precision Recall Score:",average_precision)


Fuzzy

In [0]:
X1 = df4.iloc[:,23:31]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier as RFC
estimators = [100,150,200,300,400,600,800]

test_scores = []
train_scores = []
for i in estimators:
    clf = RFC(n_estimators=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('estimators = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:

Depth = [5,10,12,15,20,25,50]
test_scores = []
train_scores = []
for i in Depth:
    clf = RFC(n_estimators=100,max_depth=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('Depth = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:
#optimum depth = 11, optimum components = 400:
clf = RFC(n_estimators=400,max_depth=11,n_jobs=-1)
clf.fit(X1_train,y1_train)
y_pred_rf = clf.predict(X1_test)

In [0]:
confusion_matrix_rf = confusion_matrix(y1_test, y_pred_rf,labels = [1,0])
print(confusion_matrix_rf)

accuracy_rf = accuracy_score(y1_test,y_pred_rf)
print(accuracy_rf)

In [0]:
# RANDOM FOREST METRICS:- plot the confusion matrix, Average precision Recall, F1 score-
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from inspect import signature


average_precision = average_precision_score(y1_test,y_pred_rf)    
f1_score_rf = f1_score(y1_test,y_pred_rf)
print("F1 Score of Random Forest classifier is:",f1_score_rf)
print("Confusion Matrix for Random Forest is:",confusion_matrix_rf)
print("precision score:",precision_score(y1_test, y_pred_rf))
print("Recall:",recall_score(y1_test,y_pred_rf))
print("Accuracy:",accuracy_rf)
print("Average Precision Recall Score:",average_precision)


Fuzzy + Word2Vec Embeddings

In [0]:
rng = list(range(1,15)) + list(range(23,30))
X1 = df4.iloc[:,rng]
y1 = df4['is_duplicate']
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.13, random_state=42)

In [0]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier as RFC
estimators = [100,150,200,300,400,600,800]

test_scores = []
train_scores = []
for i in estimators:
    clf = RFC(n_estimators=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('estimators = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:

Depth = [5,10,12,15,20,25,50]
test_scores = []
train_scores = []
for i in Depth:
    clf = RFC(n_estimators=100,max_depth=i,n_jobs=-1)
    clf.fit(X1_train,y1_train)
    predict_y = clf.predict_proba(X1_train)
    log_loss_train = log_loss(y1_train, predict_y, eps=1e-15)
    train_scores.append(log_loss_train)
    predict_y = clf.predict_proba(X1_test)
    log_loss_test = log_loss(y1_test, predict_y, eps=1e-15)
    test_scores.append(log_loss_test)
    print('Depth = ',i,'Train Log Loss ',log_loss_train,'Test Log Loss ',log_loss_test)

In [0]:
#optimum depth = 11, optimum components = 400:
clf = RFC(n_estimators=400,max_depth=11,n_jobs=-1)
clf.fit(X1_train,y1_train)
y_pred_rf = clf.predict(X1_test)

In [0]:
confusion_matrix_rf = confusion_matrix(y1_test, y_pred_rf,labels = [1,0])
print(confusion_matrix_rf)

accuracy_rf = accuracy_score(y1_test,y_pred_rf)
print(accuracy_rf)

In [0]:
# RANDOM FOREST METRICS:- plot the confusion matrix, Average precision Recall, F1 score-
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from inspect import signature


average_precision = average_precision_score(y1_test,y_pred_rf)    
f1_score_rf = f1_score(y1_test,y_pred_rf)
print("F1 Score of Random Forest classifier is:",f1_score_rf)
print("Confusion Matrix for Random Forest is:",confusion_matrix_rf)
print("precision score:",precision_score(y1_test, y_pred_rf))
print("Recall:",recall_score(y1_test,y_pred_rf))
print("Accuracy:",accuracy_rf)
print("Average Precision Recall Score:",average_precision)
